# Learning points

- count inversions with divide and conquer
- find the closest pair of points in 2d-plane

## Count inversion in the array

Inversion defined:

- given a list $L$ with index $i < j$, then an inversion is when $L_{i} > L_{j}$
- Number of inversions are counted from the modified version of merge-sort in the merge subroutine:

    - The inversions occur in between the left and right sublists, defined here as split inversion (`split_inv`)
    - In the split inversion, an inversion happens when an element from right sublist is being selected instead of the left sublist. The number of inversion is then the rest of elements in the left sublist. As these are the inversions as defined, above.
    - Note: the left sublist sorted has index $<$ the index in the right sublist after sorted


In [82]:
# functional style to count the inversion in a list of integers

def count_inv_recur(L):
    res = {'count': 0}
    inv_sort(L, res)
    return res['count']

def inv_sort(L, res):
    
    def merge(L1, L2):
        if (len(L1) == 0): return L2
        elif (len(L2) == 0): return L1
        elif L1[0] < L2[0]: return [L1[0]] + merge(L1[1:], L2)
        else: 
            res['count'] = res['count'] + len(L1)
            return ([L2[0]] + merge(L1, L2[1:]))
    n = len(L) // 2
    if (n == 0): return L
    else: 
        return merge(inv_sort(L[0:n], res), inv_sort(L[n:], res))

assert count_inv_recur([1 ,3, 5, 2, 4, 6]) == 3
assert count_inv_recur([6, 5, 4, 3, 2, 1]) == 15
print "inversion count works"

inversion count works


In [83]:
# using merge sort divide-conquer paradigmn

def sort_and_count(L):
    if len(L) <= 1:
        return L, 0
    mid = len(L) // 2
    left, left_inv = sort_and_count(L[:mid])
    right, right_inv = sort_and_count(L[mid:])
    merged, split_inv = merge_count(left, right)
    count = left_inv + right_inv + split_inv
    return merged, count
    
def merge_count(left, right):
    result = list()
    i, j = 0, 0
    inv_count = 0
    while i < len(left) and j < len(right):
        if right[j] < left[i]:
            result.append(right[j])
            j += 1
            inv_count += (len(left) - i)
        else:
            result.append(left[i])
            i += 1
    result += left[i:]
    result += right[j:]
    return result, inv_count

assert sort_and_count([1 ,3, 5, 2, 4, 6])[1] == 3
assert sort_and_count([6, 5, 4, 3, 2, 1])[1] == 15
print "inversion count works"

inversion count works


## Find the closest pair

- Prior knowledge, see [wiki](https://en.wikipedia.org/wiki/Closest_pair_of_points_problem)
- Input: a set of points ${P_1, P_2, ..., P_n}$ in a 2D plane
- Output: a pair of points $p^*, q^*$ of distinct points that minimize $d(p, q)$ over $p,q \in P$
- The distance between two points, defined as a $Eucledian$ distance, given $P_i = (x_i, y_i), and P_j = (x_j, y_j)$
    
    $$d(P_i, P_j) = \sqrt{(x_i - x_j)^2 + (y_i - y_j)^2}$$
- Assumption: All points have distinct x- and y-coordinates, approached with divide-and-conquer paradigm
- Pseudocode:

    1. make copies of points, i.e, Px and Py
        Px is sorted by x-coordinate and Py is sorted by y-coordinate
    2. Divide the `Points` into half, Q (left) and R (right) sides to form Q_x, Q_y, and R_x, R_y
    3. (p1, q1, d1) = closest_pair(Q_x, Q_y) from the left side
    4. (p2, q2, d2) = closest_pair(R_x, R_y) from the right side
    5. (p3, q3, d3) = closest_pair from points between the left and right sides with a delta (from left and right)
    6. return the closest pair of (p1, q1), (p2, q2), (p3, q3) with the smallest m{1,2,3}

    

In [84]:
import math

def dist(p1, p2):
    return math.sqrt( (p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2 )

def find_closest_pair(Points):
    P_x = sorted(Points, key = lambda p: p[0])
    P_y = sorted(Points, key = lambda p: p[1])
    p1, p2, m = closest_pair(P_x, P_y)
    return sorted([p1, p2], key = lambda p: p[0])

def closest_pair(P_x, P_y):
    num_points = len(P_x)
    if num_points <= 3: 
        return brute_force(P_x)
    mid = num_points // 2
    Q_x = P_x[:mid]
    R_x = P_x[mid:]
    
    x_bar = P_x[mid][0]
    Q_y, R_y = [], []
    for p in P_y:
        if p[0] <= x_bar:
            Q_y.append(p)
        else:
            R_y.append(p)
    p1, q1, d1 = closest_pair(Q_x, Q_y)
    p2, q2, d2 = closest_pair(R_x, R_y)
    if d1 <= d2:
        delta = d1
        bestpair = p1, q1
    else:
        delta = d2
        bestpair = p2, q2
    p3, q3, d3 = closest_split_pair(P_x, P_y, delta)
    
    if delta <= d3:
        return bestpair[0], bestpair[1], delta
    else:
        return p3, q3, d3

def closest_split_pair(P_x, P_y, delta):
    num_points = len(P_x)
    x_bar = P_x[num_points // 2][0]
    s_y = [ x for x in P_y if x_bar - delta <= x[0] <= x_bar + delta ]
    best = delta
    best_pair = None, None
    for i in xrange(len(s_y) -1):
        for j in xrange(i+1, min(i + 7, len(s_y))):
            p, q = s_y[i], s_y[j]
            dst = dist(p, q)
            if dst < best:
                best_pair = p, q
                best = dst
    return best_pair[0], best_pair[1], best
    
    
def brute_force(P):
    P = [p for p in P if p is not None]
    num_points = len(P)
    min_d = dist(P[0], P[1])
    if num_points <= 2: 
        return P[0], P[1], min_d
    else:
        for i in xrange(num_points):
            for j in xrange(i+1, num_points):
                if i == 0 and j == 1: continue
                else:
                    d = dist(P[i], P[j])
                    if d < min_d:
                        min_d = d
                        p1, p2 = P[i], P[j]
    return p1, p2, min_d
        
    


assert find_closest_pair([(1, 1), (1, 5), (2, 3), (3, 2), (5, 5), (5, 3)]) == [(2, 3), (3, 2)]
assert find_closest_pair([(1, 1), (1, 5), (2, 3), (3, 2), (5, 5), (5, 3), (4, 4)]) == [(4, 4), (5, 3)]
print "closest pair works"

closest pair works
